<a href="https://colab.research.google.com/github/karjain/Anti_deepFake/blob/main/Predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)
%cd /content/drive

Mounted at /content/drive
/content/drive


In [6]:
import numpy as np
import time

import PIL.Image as Image
import matplotlib.pylab as plt

import tensorflow as tf
#import tensorflow_hub as hub

import os
import argparse
from os.path import join
import cv2
import dlib
#import torch
#import torch.nn as nn
from PIL import Image as pil_image
from tqdm import tqdm

In [4]:
!ls

sample_data


In [7]:
reload_model = tf.keras.models.load_model('/content/drive/MyDrive/Fake_dect/model.h5')